<a href="https://colab.research.google.com/github/mapsguy/programming-gemini/blob/main/native_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#step 1: install/upgrade the latest genai SDK
%pip install google-genai --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.7/217.7 kB 8.4 MB/s eta 0:00:00


In [5]:
#import the genai library
from google import genai
from google.genai import types

In [3]:
#step 2: AIStudio: read the api key from the user data
from google.colab import userdata
client = genai.Client(api_key=userdata.get("GEMINI_API_KEY"))

#If you want to read from environment keys
#import os
#client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [4]:
model_name = "models/gemini-2.5-flash-preview-05-20"

In [40]:
#Google Search

#Define the grounding tool
grounding_tool = types.Tool(
    google_search=types.GoogleSearch()
)

# Configure generation settings
config = types.GenerateContentConfig(
    tools=[grounding_tool]
)

# Make the request
response = client.models.generate_content(
    model=model_name,
    contents="What medals did USA win in 2024 Olympics?",
    config=config,
)

# Print the grounded response
print(response.text)

The United States won a total of 126 medals at the 2024 Summer Olympics in Paris, topping the medal table for the fourth consecutive Summer Games and 19th time overall. This impressive haul included 40 gold medals, 44 silver medals, and 42 bronze medals.


In [7]:
#Using code execution in chat

chat = client.chats.create(
    model=model_name,
    config=types.GenerateContentConfig(
        tools=[types.Tool(code_execution=types.ToolCodeExecution)]
    ),
)

response = chat.send_message("I have a physics question for you.")
print(response.text)

response = chat.send_message(
    "What is the right hand rule for angular momentum?"
    "Generate a practice problem for this question."
)

for part in response.candidates[0].content.parts:
    if part.text is not None:
        print(part.text)
    if part.executable_code is not None:
        print(part.executable_code.code)
    if part.code_execution_result is not None:
        print(part.code_execution_result.output)

Okay, I'm ready. Please tell me your physics question!
The right-hand rule is a mnemonic to determine the direction of the angular momentum vector. Angular momentum ($\vec{L}$) is a vector quantity, and its direction is perpendicular to both the position vector ($\vec{r}$) and the linear momentum vector ($\vec{p}$) (or the angular velocity vector ($\vec{\omega}$)).

### The Right-Hand Rule for Angular Momentum ($\vec{L} = \vec{r} \times \vec{p}$ or $\vec{L} = I\vec{\omega}$)

There are a couple of ways to apply the right-hand rule for angular momentum, depending on how you're conceptualizing it:

1.  **Using $\vec{L} = \vec{r} \times \vec{p}$ (Cross Product Method):**
    *   Point the fingers of your right hand in the direction of the position vector $\vec{r}$ (from the origin or pivot point to the object).
    *   Curl your fingers towards the direction of the linear momentum vector $\vec{p}$ (the direction of motion).
    *   Your extended thumb will point in the direction of the an

In [35]:
#Defining a function declaration
get_weather_function = {
    "name": "get_weather",
    "description": "Retrieves the weather on a given date.",
    "parameters": {
        "type": "object",
        "properties": {
            "skies": {
                "type": "string",
                "description": "The sky forecast for that day.",
            },
            "temperature": {
                "type": "string",
                "description": "The temperature forecast for that day.",
            },
        },
        "required": ["skies", "temperature"],
    },
}


In [36]:
#configure
tools = types.Tool(function_declarations=[get_weather_function])
config = types.GenerateContentConfig(tools=[tools])

In [37]:
contents = [
    types.Content(
        role="user", parts=[types.Part(text="Report the weather forecast for 07/21/2025. Skies are cloudy, and temperature is between 60-70 degrees.")]
    )
]

In [38]:
response = client.models.generate_content(
    model=model_name,
    contents=contents,
    config=config,
)

In [39]:
# Checking for a function call
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")
    #  In a real app, you would call your function here:
    #  result = schedule_meeting(**function_call.args)
else:
    print("No function call found in the response.")
    print(response.text)

Function to call: get_weather
Arguments: {'skies': 'cloudy', 'temperature': '60-70 degrees'}
